In [2]:
import torch
from datasets.download_data import download_data
from datasets.consts import Dataset, DatasetType
from models.feature_extractor import get_pretrained_model, get_transform
import torchvision
import torch
from models.CAML import CAML
import learn2learn as l2l
import tqdm
import wandb

/home/lszarejko/.pyenv/versions/few-shot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
# device = "cpu" # Due to CUDA error

device(type='cuda', index=0)

In [4]:
wandb.init(project="few-shot-learning", config={"architecture": "CAML", "dataset": "mini-imagenet"})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: szaryvip (mgr-few-shot). Use `wandb login --relogin` to force relogin


In [6]:
train_transform, test_transform = get_transform("timm/vit_base_patch16_clip_224.laion2b_ft_in12k_in1k")
train_transform = torchvision.transforms.Compose(train_transform.transforms[:-2])
test_transform = torchvision.transforms.Compose(test_transform.transforms[:-2])
train = download_data(Dataset.MINI_IMAGENET, DatasetType.TRAIN, transform=train_transform)
valid = download_data(Dataset.MINI_IMAGENET, DatasetType.VAL, transform=test_transform)
test = download_data(Dataset.MINI_IMAGENET, DatasetType.TEST, transform=test_transform)

In [7]:
fe = get_pretrained_model("timm/vit_base_patch16_clip_224.laion2b_ft_in12k_in1k")
model = CAML(feature_extractor=fe, fe_dim=768, fe_dtype=torch.float32, train_fe=False, encoder_size='tiny', device=device).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

: 

In [ ]:
train_dataset = l2l.data.MetaDataset(train)
transforms = [
    l2l.data.transforms.FusedNWaysKShots(train_dataset, n=5, k=1),
    l2l.data.transforms.LoadData(train_dataset),
    l2l.data.transforms.RemapLabels(train_dataset),
]

train_tasksets = l2l.data.Taskset(train_dataset, task_transforms=transforms, num_tasks=1000)
train_loader = torch.utils.data.DataLoader(train_tasksets, shuffle=True)

In [ ]:
valid_dataset = l2l.data.MetaDataset(valid)
transforms = [
    l2l.data.transforms.FusedNWaysKShots(valid_dataset, n=5, k=1),
    l2l.data.transforms.LoadData(valid_dataset),
    l2l.data.transforms.RemapLabels(valid_dataset),
]
valid_tasksets = l2l.data.Taskset(valid_dataset, task_transforms=transforms, num_tasks=100)
valid_loader = torch.utils.data.DataLoader(valid_tasksets, shuffle=True)

In [ ]:
test_dataset = l2l.data.MetaDataset(test)
transforms = [
    l2l.data.transforms.FusedNWaysKShots(test_dataset, n=5, k=1),
    l2l.data.transforms.LoadData(test_dataset),
    l2l.data.transforms.RemapLabels(test_dataset),
]
test_tasksets = l2l.data.Taskset(test_dataset, task_transforms=transforms, num_tasks=100)
test_loader = torch.utils.data.DataLoader(test_tasksets, shuffle=True)

In [ ]:
model.train()
epochs = 5
best_val_acc = 0
for epoch in tqdm.tqdm(range(epochs)):
    avg_loss = 0.0
    avg_acc = 0.0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        X, labels = batch
        shot=1
        batch_size, way, channels, height, width = X.shape
        X = X.view(batch_size * way, channels, height, width)
        labels = labels.view(-1)
        X, labels = X.to(device), labels.to(device)
        logits = model(X, labels[:(way-1)*shot], way=(way-1), shot=shot)
        loss = criterion(logits, labels[(way-1)*shot:])
        loss.backward()
        optimizer.step()
        
        loss_value = loss.item()
        _, max_index = torch.max(logits, 1)
        
        acc = 100 * torch.sum(torch.eq(max_index, labels[(way-1)*shot:])).item() / labels[(way-1)*shot:].shape[0]
        avg_acc += acc
        avg_loss += loss_value
    avg_acc = avg_acc / (i + 1)
    avg_loss = avg_loss / (i + 1)
    print(f"Epoch {epoch} - Loss: {avg_loss}, Acc: {avg_acc}")
    wandb.log({"train_acc": acc, "train_loss": loss})
    
    torch.save(model.state_dict(), "model.pth")
    wandb.save("model.pth")
    
    # Validation
    avg_acc = 0.0
    avg_loss = 0.0
    for i, batch in enumerate(valid_loader):
        X, labels = batch
        shot=1
        batch_size, way, channels, height, width = X.shape
        X = X.view(batch_size * way, channels, height, width)
        labels = labels.view(-1)
        X, labels = X.to(device), labels.to(device)
        logits = model(X, labels[:(way-1)*shot], way=(way-1), shot=shot)
        loss = criterion(logits, labels[(way-1)*shot:])
        loss_value = loss.item()
        _, max_index = torch.max(logits, 1)
        
        acc = 100 * torch.sum(torch.eq(max_index, labels[(way-1)*shot:])).item() / labels[(way-1)*shot:].shape[0]
        avg_acc += acc
        avg_loss += loss_value
    avg_acc = avg_acc / (i + 1)
    avg_loss = avg_loss / (i + 1)
    print(f"Validation - Loss: {avg_loss}, Acc: {avg_acc}")
    wandb.log({"valid_acc": acc, "valid_loss": loss})
    best_val_acc = max(best_val_acc, avg_acc)
    
    if avg_acc == best_val_acc:
        torch.save(model.state_dict(), "best_model.pth")
        wandb.save("best_model.pth")
    

  0%|          | 0/5 [03:05<?, ?it/s]

Epoch 0 - Loss: 1.622288194462657, Acc: 26.2


Error: You must call wandb.init() before wandb.log()

In [ ]:
wandb.finish()

acc,▁
loss,▁
acc,0
loss,1.41545
